In [4]:
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.3/436.3 kB 2.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 770.0 kB/s eta 0:00:00 0:00:01
  Attempting uninstall: packaging
    Found existing installation: packaging 20.1
    Uninstalling packaging-20.1:
      Successfully uninstalled packaging-20.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
pytest-astropy 0.8.0 requires pytest-filter-subpackage>=0.1, which is not installed.


In [ ]:
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '/Users/candide/Acads/Programming Practise/reddit_screen')

from boilerplate import *
pd.options.display.max_rows = 500
pd.options.display.max_columns = 100

In [ ]:
# set filepath to the given file
FILEPATH = "/Users/candide/Acads/Programming Practise/reddit_screen/aml_hws/vehicles.csv"

# load the data - https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html 
data = pd.read_csv(FILEPATH, sep=",", header='infer', index_col=None)

In [ ]:
# see what the data looks like
data.head(10)

### Get an insight into the dataset

In [ ]:
# sample dataset in case it's massive
data = data.sample(100000, random_state=42)

##### TARGET

In [ ]:
# what is the target column
TARGET_COL = "price"

In [ ]:
# visualise the distribution of the target for potential outlier values

# if it's continuous
plt.boxplot(data[TARGET_COL], vert=False)

# if it's discrete
#data[TARGET_COL].hist()

In [ ]:
data.price.describe()

In [ ]:
data = data[data.price < 1000000000]

In [ ]:
# split into categorical and continuous features broadly
X, y = split_df_x_y(data, "price")

In [ ]:
# get a quick view into the data types of features
summarize_data(X)

In [ ]:
# stats on continuous features
X.describe() # by default only describes continous features

In [ ]:
# drop columns for continuous features which seem empty/irrelevant from the stats
X_cont_dropped = X.drop(["county", "id"], axis=1)

In [ ]:
# separate continuous vs categorical features broadly
cont_features, cat_features = separate_cont_cat(X_cont_dropped)

In [ ]:
# visualize continuous features
visualize_feature_distributions_rough(X_cont_dropped, cont_features)

In [ ]:
# get stats on categorical features
cat_stats = categorical_variables_stats(X_cont_dropped, cat_features)
cat_stats

In [ ]:
# drop columns for categorical features which seem empty/irrelevant from the stats
X_cont_cat_dropped = X_cont_dropped.drop(["url", "posting_date", "description", "image_url", "region", "VIN"], axis=1)

In [ ]:
# fitler continuous vs categorical features again
cont_features, cat_features = separate_cont_cat(X_cont_cat_dropped)

In [ ]:
X_cont_cat_dropped.columns

In [ ]:
X_cont_cat_dropped.info()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_cont_cat_dropped, y, test_size=0.2)

In [ ]:
cat_preprocessing = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='NA'),
    OneHotEncoder(handle_unknown='ignore'))
cont_preprocessing = make_pipeline(
    SimpleImputer(strategy="median"),
    MinMaxScaler())

In [ ]:
cat_preprocessing.fit(X_train[cat_features])

In [ ]:
cont_preprocessing.fit(X_train[cont_features])

In [ ]:
len(X_train.columns) == len(cat_features) + len(cont_features)

In [ ]:
col_transformer = make_column_transformer(
    (cat_preprocessing, cat_features),
    remainder=cont_preprocessing
)

In [ ]:
np.isnan(col_transformer.fit_transform(X_train).toarray())

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf_pipe = make_pipeline(col_transformer, RandomForestRegressor())

In [ ]:
rf_pipe.fit(X_train, y_train.values)

In [ ]:
lr_pipe.score(X_test, y_test)

In [ ]:
cross_val_score(lr_pipe, X_train, y_train)